In [8]:
import pandas as pd 
import pydeck as pdk
# import geopandas as gpd
# from shapely.geometry import LineString
import matplotlib.pyplot as plt
# import contextily as ctx
import networkx as nx
import copy
from tqdm import tqdm
import re

import os

In [24]:
df = pd.read_csv("../data/MTA_Subway_Origin-Destination_Ridership_Estimate__2024_20251020.csv")

C:\Users\meenakshi\AppData\Local\Temp\ipykernel_11068\2658214266.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/MTA_Subway_Origin-Destination_Ridership_Estimate__2024_20251020.csv")


In [25]:
df.head()

,Day of Week,sum_estimated_average_ridership,Origin Station Complex ID,Origin Station Complex Name,Origin Latitude,Origin Longitude,Destination Station Complex ID,Destination Station Complex Name,Destination Latitude,Destination Longitude,Year,Month,Timestamp
0,Monday,0.2993,430,"Simpson St (2,5)",40.824073,-73.893064,461,"Queensboro Plaza (7,N,W)",40.750582,-73.940202,2024,8,08/05/2024 07:00:00 AM
1,Monday,2.266,301,181 St (1),40.849505,-73.933596,316,50 St (1),40.761728,-73.983849,2024,8,08/05/2024 04:00:00 AM
2,Monday,0.5905,246,Avenue I (F),40.625322,-73.976127,28,Union St (R),40.677316,-73.983110,2024,8,08/05/2024 09:00:00 AM
3,Monday,0.3052,395,103 St (6),40.790600,-73.947478,617,"Atlantic Av-Barclays Ctr (B,D,N,Q,R,2,3,4,5)",40.684162,-73.977789,2024,8,08/05/2024 06:00:00 AM
4,Monday,0.65,403,33 St (6),40.746081,-73.982076,379,Mosholu Pkwy (4),40.879750,-73.884655,2024,8,08/05/2024 02:00:00 AM


In [ ]:
df["Timestamp"] = pd.to_datetime(df["Timestamp"])
df["DayType"] = df["Timestamp"].dt.dayofweek.map(lambda x: "Weekday" if x < 5 else "Weekend")

# clean commas
# df["sum_estimated_average_ridership"] = pd.to_numeric(
#     df["sum_estimated_average_ridership"].astype(str).str.replace(",", ""), errors="coerce"
# )

df["sum_estimated_average_ridership"] = pd.to_numeric(
    df["sum_estimated_average_ridership"].astype(float))

# group and aggregate mean while keeping representative metadata columns
agg = (
    df.groupby(["DayType", "Origin Station Complex ID", "Destination Station Complex ID"], as_index=False)
      .agg({
          "sum_estimated_average_ridership": "mean",
          "Origin Station Complex Name": "first",
          "Destination Station Complex Name": "first",
          "Origin Latitude": "first",
          "Origin Longitude": "first",
          "Destination Latitude": "first",
          "Destination Longitude": "first"
      })
)

# rename column to reflect mean
agg = agg.rename(columns={"sum_estimated_average_ridership": "mean_estimated_average_ridership"})




In [57]:
agg.to_csv("../data/weekday_weekend_aggregated.csv", index=False)

In [56]:
print(len(agg))

332140
